<a href="https://colab.research.google.com/github/felipesolferini/combustioncourse/blob/main/Combustion_coke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an analysis of coal combustion in a general furnace.



The first cell are the inputs/parameters of the process.

In [ ]:
# Code created by Felipe Solferini de Carvalho - 17/04/2024

# Inputs:

alpha = 1.17  # Equivalence ratio of combustion gases on the furnace

frac = 1  # Mass fraction of input 1 (1-X), a value between 0 and 1

# Temperatures of the process
T_1 = 25 + 273  # [K] Ambient temperature
T_2 = 2000  # [K] Adiabatic flame temperature guess
T_3 = 25 + 273  # [K] Preheating temperature of the combustion air

# Equivalence ratio of the process
CI = 1  # Mass fraction of converted carbon, a value between 0 and 1 (normally this value ranges from 0.95 to 1, considered 1 for the adiabatic flame temperature)
gama = 1  # Fraction of heat loss to the walls, a value between 0 and 1 (normally this value ranges from 0.93 to 0.97, considered 1 for the adiabatic flame temperature)

# Composition of feedstock 1. (coke)
C1 = 83.7 # [%]
H1 = 3.6 # [%]
N1 = 1.4 # [%]
S1 = 4.61 # [%]
O1 = 3.13 # [%]
ashes1 = 0.2 # [%]

moisture1 = 3.40  # [%]
LHV1 = 4.184*809600 # Lower heating value of fuel 1 [J/100g]

# Composition of biomass 2 (hydrogen, or other)
C2 = 0 # [%]
H2 = 100 # [%]
N2 = 0 # [%]
S2 = 0 # [%]
O2 = 0 # [%]
ashes2 = 0 # [%]

moisture2 = 0  # [%]
LHV2 = 12000000 # Lower heating value of fuel 2 [J/100g]


# BEGINNING OF CALCULATIONS
# DEFINITIONS

# Average composition of the feedstock
C = frac * C1 + (1 - frac) * C2 # [%]
H = frac * H1 + (1 - frac) * H2 # [%]
N = frac * N1 + (1 - frac) * N2 # [%]
S = frac * S1 + (1 - frac) * S2 # [%]
O = frac * O1 + (1 - frac) * O2 # [%]
ashes = frac * ashes1 + (1 - frac) * ashes2 # [%]
moisture = frac * moisture1 + (1 - frac) * moisture2 # [%]

x = C / 12
y = H
z = O / 16
k = N / 14
e = S / 32

# Reference values:

massa_mol = 100  # [g/mol]

# Universal constant of perfect gases
R = 8.314  # [kJ/kmolK]

#latent heat of water [J/mol]
cal_vap = 44012

# Number of moles of ashes (considered to be SiO2)
cin = ashes / 60

# Variation of temperature
delta_T = (T_2 - T_1)

# Calculating the moisture coefficient (w)
w = moisture / (18 * (1 - moisture / 100))

# Calculating the stoichiometric coefficient (m)

m = x + y/4 - z/2 + e

# Sensible Enthalpy Calculations

# Carbon Dioxide
def CO2_heat(T_i,T_j):
    return (-3.73589 * (T_i - T_j) + 2.0354 * ((T_i ** 1.5) - (T_j ** 1.5)) - 0.020516 * ((T_i ** 2) - (T_j** 2)) + \
            8.067 * 10 ** (-7) * ((T_i ** 3) - (T_j ** 3)))

# Water
def h2o_heat(T_i,T_j):
    return 34.392 * (T_i - T_j) + 0.0003138 * ((T_i ** 2) - (T_j ** 2)) + 0.00000187 * ((T_i ** 3) - (T_j ** 3))

# Sulfur
def so2_heat(T_i,T_j):
    return 32.217 * (T_2 - T_1) + 0.01109 * (T_2 ** 2 - T_1 ** 2) - 0.00000116 * (T_2 ** 3 - T_1 ** 3)

# Oxygen
def o2_heat(T_i,T_j):
    return 4.184 * (0.000129 * (T_i ** 2 - T_j ** 2) + 8.27 * (T_i - T_j) + (187700 / T_i) - (187700 / T_j))

# Nitrogen
def n2_heat(T_i,T_j):
    return 27.196 * (T_i - T_j) + 0.002092 * ((T_i ** 2) - (T_j ** 2))

# Equations for Silica
def si_heat(T_i,T_j):
    return 45.480 * (T_i - T_j) + 0.018226 * (T_i ** 2 - T_j ** 2) + 1009181 * ((1 / T_i) - (1 / T_j)) + 45.815 * (T_i - T_j) + 0.011506 * (T_i ** 2 - T_j ** 2)

# Heating value of the fuel (J/100g)
#I_c = 32760 * C + 121540 * H - 14880 * O - 1800 * N + 4420 * S + 4420 * ((H) ** 0.3)
I_c = (LHV1*frac)+(1-frac)*LHV2

print("Heating value of the fuel:", I_c, 'J/ 100g')

def equation(T2_guess, *args):
    # Unpacking the arguments
    T3, T1, I_c, cal_vap, w, m, alpha, x, y, e, k, gama, CI, cin = args

    # Calculating the the conservation of energy (energy input - energy output)
    input_energy = I_c*gama*CI - cal_vap * w + m * alpha * o2_heat(T3, T1) + 3.76 * m * alpha * n2_heat(T3, T1)
    output_energy = (x * CO2_heat(T2_guess, T1) + (y / 2 + w) * h2o_heat(T2_guess, T1) + e * so2_heat(T2_guess, T1) +
        m * (alpha - 1) * o2_heat(T2_guess, T1) + (3.76 * m * alpha + k) * n2_heat(T2_guess, T1) + cin*si_heat(T2_guess,T1))

    return input_energy - output_energy

# Implementing the Newton-Raphson method
def newton_raphson(initial_guess, *args, tol=1e-6, max_iter=100):
    # Initializing iteration counter and guess value
    iter_count = 0
    T2_guess = initial_guess

    while True:
        # Calculate the value of the function and its derivative at the current guess
        f_value = equation(T2_guess, *args)

        # Check for convergence
        if abs(f_value) < tol or iter_count >= max_iter:
            break

        # Update the guess value using Newton-Raphson iteration formula
        # Assuming the derivative function is available as `equation_derivative`
        # T2_guess -= f_value / equation_derivative(T2_guess, *args)

        # If the derivative function is not available, approximate it using numerical differentiation
        # You can use methods like central difference approximation
        delta = 1e-6  # Small perturbation for numerical differentiation
        f_derivative = (equation(T2_guess + delta, *args) - equation(T2_guess - delta, *args)) / (2 * delta)
        T2_guess -= f_value / f_derivative

        # Increment iteration counter
        iter_count += 1

    return T2_guess

# Call the Newton-Raphson function to solve for T2
T2_solution = newton_raphson(T_2, T_3, T_1, I_c, cal_vap, w, m, alpha, x, y, e, k, gama,CI,cin)

Stoich_Air_fuel_ratio = 10*4.76*m/44.6428

Air_fuel_ratio = 10*4.76*m*alpha/44.6428

print("Adiabatic flame temperature:", T2_solution, "K")
print("Stoichiometric Air/fuel ratio:", Stoich_Air_fuel_ratio,"Nm³/kgfuel")
print("Operational Air/fuel ratio:", Air_fuel_ratio,"Nm³/kgfuel")


Heating value of the fuel: 3387366.4 J/ 100g
Adiabatic flame temperature: 2319.5585440047457 K
Stoichiometric Air/fuel ratio: 8.445964410834446 Nm³/kgfuel
Operational Air/fuel ratio: 9.881778360676302 Nm³/kgfuel
